In [12]:
import selenium
import os 
from bs4 import BeautifulSoup as bs
from urllib import request as rq
from urllib.request import urlopen
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import warnings
warnings.filterwarnings('ignore')

### 신세계 원본 데이터 불러오기

In [2]:
shin_df = pd.read_pickle('신세계wine_delete.pkl')
shin_df

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,바디,와인설명,수상,와이너리이름,와이너리설명,와이너리특징
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",당도 1,산도 2,바디 4,밝게 빛나는 진한 적색을 띠고 있으며 붉은 딸기 류의 풍부한 향과 스파이시한 노트가...,"2011 : Robert Parker 88점, Bettane + Desseauve ...",이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",당도 1,산도 3,바디 3,"비오니에 포도품종 특유의 살구, 복숭아 등의 신선한 과일 향과 향긋한 아카시아 꽃 ...",NaN,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,바디 5,"붉은 딸기, 라스베리, 체리 등의 진한 향과 오크에서 배어나는 토스트 향, 볶은 커...",2013 : Wine Spectator 93점2009 : Robert Parker ...,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,바디 5,"잘 익은 자두와 블랙베리, 라즈베리 잼처럼 진하고 응축된 과일 향과 섬세한 향신료 ...",2012 : Wine Spectator 95점 2011 : Wine Spectato...,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
4,이기갈 꽁드리유,E.Guigal Condrieu,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",당도 1,산도 3,바디 4,"살구, 복숭아, 시트러스 계열의 달콤하고 새콤한 과일 향이 주된 아로마를 이룬다. ...","2013 : Robert Parker 92점, Gault & Millau 16.5/...",이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,까테나 자파타 까테나 알타 까베르네 소비뇽,Catena Zapata Catena Alta Cabernet Sauvignon,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,까베르네 소비뇽 100%,750ml,"치킨 커틀렛, 치즈 리조또, 숙성이 잘 된 치즈",당도 1,산도 3,바디 5,까테나 알타는 까테나 가문에서 관리해 온 밭 중에서도 최상의 조화를 이룰 수 있는 ...,2019: Robert Parker 93점2018: Robert Parker 92점...,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...
893,까테나 자파타 까테나 알타 말벡,Catena Zapata Catena Alta Malbec,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말백 100%,750ml,"달콤한 육포, 스테이크, 훈연 소시지, 치즈",당도 1,산도 3,바디 4,까테나 알타는 까테나 가문에서 관리해 온 밭 중에서도 최상의 조화를 이룰 수 있는 ...,2019: Robert Parker 94점2018: James Suckling 95...,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...
894,까테나 자파타 니카시아 말벡,Catena Zapata Nicasia Malbec,까테나 자파타 최고도 빈야드에서 생산되는 100% 말벡 와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,750ml,"양갈비, 로스트비프, 트러플 치즈 파니니, 머쉬룸 파스타",당도 1,산도 3,바디 5,"짙은 레드 계열의 바이올렛 컬러로 잘 숙성된 검은 과일, 달달한 스파이스 등 까떼나...","2018: Robert Parker 98점, James Suckling 96점201...",까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...
895,까테나 자파타 아르헨티노 말벡,Catena Zapata Argentino Malbec,"열정의 나라, 아르헨티나의 프리미엄 와인",레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,750ml,"치즈, 육즙이 풍부한 스테이크, 양고기, 블루 치즈",당도 1,산도 4,바디 5,"깊고 어두운 자주빛을 띠고, 라즈베리와 블랙베리의 향이 진한 와인으로 훌륭한 밸런스...","2019 James Suckling 99점, Robert Parker 96점2018...",까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...


In [3]:
shin_df

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,바디,와인설명,수상,와이너리이름,와이너리설명,와이너리특징
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",당도 1,산도 2,바디 4,밝게 빛나는 진한 적색을 띠고 있으며 붉은 딸기 류의 풍부한 향과 스파이시한 노트가...,"2011 : Robert Parker 88점, Bettane + Desseauve ...",이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",당도 1,산도 3,바디 3,"비오니에 포도품종 특유의 살구, 복숭아 등의 신선한 과일 향과 향긋한 아카시아 꽃 ...",NaN,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,바디 5,"붉은 딸기, 라스베리, 체리 등의 진한 향과 오크에서 배어나는 토스트 향, 볶은 커...",2013 : Wine Spectator 93점2009 : Robert Parker ...,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,바디 5,"잘 익은 자두와 블랙베리, 라즈베리 잼처럼 진하고 응축된 과일 향과 섬세한 향신료 ...",2012 : Wine Spectator 95점 2011 : Wine Spectato...,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
4,이기갈 꽁드리유,E.Guigal Condrieu,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",당도 1,산도 3,바디 4,"살구, 복숭아, 시트러스 계열의 달콤하고 새콤한 과일 향이 주된 아로마를 이룬다. ...","2013 : Robert Parker 92점, Gault & Millau 16.5/...",이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,까테나 자파타 까테나 알타 까베르네 소비뇽,Catena Zapata Catena Alta Cabernet Sauvignon,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,까베르네 소비뇽 100%,750ml,"치킨 커틀렛, 치즈 리조또, 숙성이 잘 된 치즈",당도 1,산도 3,바디 5,까테나 알타는 까테나 가문에서 관리해 온 밭 중에서도 최상의 조화를 이룰 수 있는 ...,2019: Robert Parker 93점2018: Robert Parker 92점...,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...
893,까테나 자파타 까테나 알타 말벡,Catena Zapata Catena Alta Malbec,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말백 100%,750ml,"달콤한 육포, 스테이크, 훈연 소시지, 치즈",당도 1,산도 3,바디 4,까테나 알타는 까테나 가문에서 관리해 온 밭 중에서도 최상의 조화를 이룰 수 있는 ...,2019: Robert Parker 94점2018: James Suckling 95...,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...
894,까테나 자파타 니카시아 말벡,Catena Zapata Nicasia Malbec,까테나 자파타 최고도 빈야드에서 생산되는 100% 말벡 와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,750ml,"양갈비, 로스트비프, 트러플 치즈 파니니, 머쉬룸 파스타",당도 1,산도 3,바디 5,"짙은 레드 계열의 바이올렛 컬러로 잘 숙성된 검은 과일, 달달한 스파이스 등 까떼나...","2018: Robert Parker 98점, James Suckling 96점201...",까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...
895,까테나 자파타 아르헨티노 말벡,Catena Zapata Argentino Malbec,"열정의 나라, 아르헨티나의 프리미엄 와인",레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,750ml,"치즈, 육즙이 풍부한 스테이크, 양고기, 블루 치즈",당도 1,산도 4,바디 5,"깊고 어두운 자주빛을 띠고, 라즈베리와 블랙베리의 향이 진한 와인으로 훌륭한 밸런스...","2019 James Suckling 99점, Robert Parker 96점2018...",까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...


# 크롤링

## vivino: 도수, 탄닌, flavor 크롤링

In [79]:
import re
import numpy as np

options = webdriver.ChromeOptions()
options.add_argument('window_size=1920,1080')
driver = webdriver.Chrome('C:/cakd7/tool/chromedriver.exe', options = options)
driver.maximize_window()

url = 'https://www.vivino.com/e-guigal-cotes-du-rhone-rouge/w/7773'
driver.get(url)


body = driver.find_element(By.CSS_SELECTOR, 'body')
for i in range(6):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

time.sleep(3)


# 크롤링
# 알콜도수
dosu = driver.find_element(By.XPATH, '//*[@id="wine-page-lower-section"]/div/div[1]/div/div[2]/table/tbody').text
dosu = re.search('\d+', str(dosu))
print(dosu.group())

time.sleep(3)

# 탄닌
tannine = driver.find_elements(By.XPATH, '/html/body/div[2]/div[7]/div/div[1]/div/div[4]/div/div[2]/div[1]/table/tbody/tr')
for tan in tannine:
    try:
        if len(re.findall('Tannic',str(tan.text))) == 1:
            print(re.findall('Tannic',str(tan.text)))
            
            percent = tan.find_element(By.CLASS_NAME, 'indicatorBar__progress--3aXLX').get_attribute('style')
            print(percent)
    except:
        percent = np.nan


#flavor
flavors = driver.find_elements(By.XPATH, '/html/body/div[2]/div[7]/div/div[1]/div/div[4]/div/div[3]/div/div/div/div/button/div[2]/div[1]')

for i in range(3):
    flavors[i].click()
    F1 = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div/div[2]/span').text
    print(F1)
    
    f1 = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div/div[4]').text
    f1_all = f1.split('\n')
    f_w = [f1_all[i] for i in range(len(f1_all)) if i%2 == 1]
    f_w
    print(f_w)
    time.sleep(2)
    
    driver.find_element(By.XPATH, '//*[@id="baseModal"]/div/div[1]/a').click()
    time.sleep(2)
    
    
driver.close()

13
['Tannic']
width: 15%; left: 47.9623%;
width: 15%; left: 47.9623%;
Black Fruit
['plum', 'blackberry', 'dark fruit', 'black fruit', 'black cherry', 'blackcurrant']
Oaky
['oak', 'vanilla', 'tobacco', 'chocolate', 'cedar', 'clove']
Earthy
['earthy', 'leather', 'smoke', 'minerals', 'mushroom', 'stone']


## FOR문 만들기

In [6]:
shin_df['도수'] = np.nan
shin_df['탄닌'] = np.nan
shin_df['아로마1_대분류'] = np.nan
shin_df['아로마1_소분류'] = np.nan
shin_df['아로마2_대분류'] = np.nan
shin_df['아로마2_소분류'] = np.nan
shin_df['아로마3_대분류'] = np.nan
shin_df['아로마3_소분류'] = np.nan
len(shin_df.columns)

24

In [56]:
shin_df[shin_df['영어이름']=='Meo-Camuzet Morey Saint Denis']

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,...,와이너리설명,와이너리특징,도수,탄닌,아로마1_대분류,아로마1_소분류,아로마2_대분류,아로마2_소분류,아로마3_대분류,아로마3_소분류
159,메오 까뮈제 모레 생 드니,Meo-Camuzet Morey Saint Denis,부르고뉴의 전설적인 와인메이커 앙리 자이에의 수제자 메오 까뮈제의 와인,레드 와인,프랑스 > 부르고뉴 / 메오 까뮈제,피노누아 100%,750ml,"한우 안심구이, 오리고기, 버섯을 가미한 육류요리, 푸아그라, 치즈",당도 1,산도 3,...,도멘 메오 까뮈제(Domaine Meo-Camuzet)는 부르고뉴의 전설적인 와인메...,부르고뉴의 전설적인 와인메이커 앙리 자이에의 수제자 \r최고급 명품 와인을 소...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
shin_df.iloc[:20]

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,...,와이너리설명,와이너리특징,도수,탄닌,아로마1_대분류,아로마1_소분류,아로마2_대분류,아로마2_소분류,아로마3_대분류,아로마3_소분류
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",당도 1,산도 2,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,47.9624,Black Fruit,"plum, blackberry, dark fruit, black fruit, bla...",Oaky,"oak, vanilla, tobacco, chocolate, cedar, clove",Earthy,"earthy, leather, smoke, minerals, mushroom, stone"
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"peach, apricot, pear, apple, melon, stone fruit",Earthy,"minerals, honey, stone, earthy, smoke, ginger",Citrus,"citrus, lemon, grapefruit, lime, orange, tange..."
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.6574,Oaky,"oak, vanilla, tobacco, chocolate, clove, cedar",Black Fruit,"blackberry, plum, black fruit, black cherry, d...",Earthy,"leather, smoke, earthy, mushroom, game, minerals"
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.3458,Oaky,"oak, vanilla, chocolate, tobacco, coffee, cedar",Black Fruit,"blackberry, plum, black fruit, dark fruit, cas...",Earthy,"leather, smoke, earthy, minerals, mushroom, tar"
4,이기갈 꽁드리유,E.Guigal Condrieu,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"apricot, peach, pear, stone fruit, melon, apple",Earthy,"honey, stone, minerals, ginger, smoke, flint",Oaky,"vanilla, oak, butter, caramel, butterscotch, c..."
5,이기갈 꽁드리유 ‘라 도리안’,E.Guigal Condrieu ‘La Doriane’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,송로버섯 요리나 푸아그라 등 은은한 풍미의 음식,당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",NaN,NaN,Tree Fruit,"apricot, peach, pear, stone fruit, melon, apple",Earthy,"honey, minerals, stone, smoke, ginger, beeswax",Oaky,"oak, vanilla, butter, caramel, butterscotch, n..."
6,이기갈 생 조셉 레드,E.Guigal Saint-Joseph Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,시라 100%,750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,이기갈 샤또뇌프 뒤 빠쁘 레드,E.Guigal Chateauneuf-du-Pape Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"그르나쉬 80%, 시라 10%, 무르베드르 5%, 기타 5%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,이기갈 에르미따쥐 레드,E.Guigal Hermitage Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,시라 100%,750ml,"붉은 육류요리, 치즈, 야생 가금류 요리",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,이기갈 에르미따쥐 화이트,E.Guigal Hermitage Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"

In [57]:
options = webdriver.ChromeOptions()
options.add_argument('window_size=1920,1080')
driver = webdriver.Chrome('C:/cakd7/tool/chromedriver.exe', options = options)
driver.maximize_window()

url = 'https://www.vivino.com/'
driver.get(url)

for idx in range(159,len(shin_df)):
    # VIVINO홈페이지 와인이름 검색
    wine_name = shin_df.loc[idx, '영어이름']
    search_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/nav/div[1]/div/div/div/form/input')    
    search_bar.send_keys(wine_name)
    search_bar.send_keys(Keys.ENTER)
    
    time.sleep(6)
    
    
    # 첫번째 게시물 클릭
    search_first = driver.find_element(By.XPATH, '/html/body/div[2]/section[1]/div/div/div/div[1]/div/div[1]/div/div[2]/div[1]/span[1]/a')
    search_first.click()
    
    time.sleep(2)
    # 와인 상세설명 페이지 크롤링
    ## 스크롤 내리기
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    for i in range(5):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    time.sleep(3)


    ## 알콜도수
    try:
        dosu = driver.find_element(By.XPATH, '//*[@id="wine-page-lower-section"]/div/div[1]/div/div[2]/table/tbody').text
        dosu = re.search('\d+', str(dosu))
        shin_df.loc[idx,'도수'] = int(dosu.group())
    except:
        shin_df.loc[idx,'도수'] = np.nan

    time.sleep(3)

    ## 탄닌
    tannine = driver.find_elements(By.XPATH, '/html/body/div[2]/div[7]/div/div[1]/div/div[4]/div/div[2]/div[1]/table/tbody/tr')
    for tan in tannine:
        try:
            if len(re.findall('Tannic',str(tan.text))) == 1:
                percent = tan.find_element(By.CLASS_NAME, 'indicatorBar__progress--3aXLX').get_attribute('style')
                shin_df.loc[idx, '탄닌'] = str(percent).split('left: ')[1].split('%')[0]
        except:
            shin_df.loc[idx, '탄닌'] = np.nan


    ## flavor
    try:
        flavors = driver.find_elements(By.XPATH, '/html/body/div[2]/div[7]/div/div[1]/div/div[4]/div/div[3]/div/div/div/div/button/div[2]/div[1]')

        for i in range(3):
            flavors[i].click()
            F1 = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div/div[2]/span').text
            shin_df.iloc[idx,-6 + (i*2)] = F1

            f1 = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div/div[4]').text
            f1_all = f1.split('\n')
            f_w = ', '.join([f1_all[i] for i in range(len(f1_all)) if i%2 == 1])
            shin_df.iloc[idx,-5 + (i*2)] = f_w
            time.sleep(3)

            driver.find_element(By.XPATH, '//*[@id="baseModal"]/div/div[1]/a').click()
            time.sleep(3)    
    except:
        pass


driver.close()

In [58]:
shin_df

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,...,와이너리설명,와이너리특징,도수,탄닌,아로마1_대분류,아로마1_소분류,아로마2_대분류,아로마2_소분류,아로마3_대분류,아로마3_소분류
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",당도 1,산도 2,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,47.9624,Black Fruit,"plum, blackberry, dark fruit, black fruit, bla...",Oaky,"oak, vanilla, tobacco, chocolate, cedar, clove",Earthy,"earthy, leather, smoke, minerals, mushroom, stone"
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"peach, apricot, pear, apple, melon, stone fruit",Earthy,"minerals, honey, stone, earthy, smoke, ginger",Citrus,"citrus, lemon, grapefruit, lime, orange, tange..."
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.6574,Oaky,"oak, vanilla, tobacco, chocolate, clove, cedar",Black Fruit,"blackberry, plum, black fruit, black cherry, d...",Earthy,"leather, smoke, earthy, mushroom, game, minerals"
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.3458,Oaky,"oak, vanilla, chocolate, tobacco, coffee, cedar",Black Fruit,"blackberry, plum, black fruit, dark fruit, cas...",Earthy,"leather, smoke, earthy, minerals, mushroom, tar"
4,이기갈 꽁드리유,E.Guigal Condrieu,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"apricot, peach, pear, stone fruit, melon, apple",Earthy,"honey, stone, minerals, ginger, smoke, flint",Oaky,"vanilla, oak, butter, caramel, butterscotch, c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,까테나 자파타 까테나 알타 까베르네 소비뇽,Catena Zapata Catena Alta Cabernet Sauvignon,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,까베르네 소비뇽 100%,750ml,"치킨 커틀렛, 치즈 리조또, 숙성이 잘 된 치즈",당도 1,산도 3,...,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,NaN,42.9292,Black Fruit,"blackberry, plum, blackcurrant, black cherry, ...",Oaky,"oak, chocolate, mocha, caramel, vanilla, incense",Red Fruit,"raspberry, cherry, red fruit, strawberry, red ..."
893,까테나 자파타 까테나 알타 말벡,Catena Zapata Catena Alta Malbec,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말백 100%,750ml,"달콤한 육포, 스테이크, 훈연 소시지, 치즈",당도 1,산도 3,...,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,14.0,32.8264,Black Fruit,"blackberry, plum, blueberry, black fruit, dark...",Oaky,"oak, vanilla, chocolate, tobacco, dark chocola...",Earthy,"leather, earthy, smoke, minerals, cocoa, mushroom"
894,까테나 자파타 니카시아 말벡,Catena Zapata Nicasia Malbec,까테나 자파타 최고도 빈야드에서 생산되는 100% 말벡 와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,750ml,"양갈비, 로스트비프, 트러플 치즈 파니니, 머쉬룸 파스타",당도 1,산도 3,...,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,14.0,27.0405,Oaky,"oak, chocolate, vanilla, tobacco, coffee, dark...",Black Fruit,"blackberry, plum, black fruit, blueberry, cass...",Earthy,"leather, cocoa, earthy, mi

In [59]:
shin_df.to_csv('shin_df_vivino.csv')
shin_df.to_pickle('shin_df_vivino.pkl')

In [67]:
shin_df.head()

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,...,와이너리설명,와이너리특징,도수,탄닌,아로마1_대분류,아로마1_소분류,아로마2_대분류,아로마2_소분류,아로마3_대분류,아로마3_소분류
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",당도 1,산도 2,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,47.9624,Black Fruit,"plum, blackberry, dark fruit, black fruit, bla...",Oaky,"oak, vanilla, tobacco, chocolate, cedar, clove",Earthy,"earthy, leather, smoke, minerals, mushroom, stone"
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"peach, apricot, pear, apple, melon, stone fruit",Earthy,"minerals, honey, stone, earthy, smoke, ginger",Citrus,"citrus, lemon, grapefruit, lime, orange, tange..."
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.6574,Oaky,"oak, vanilla, tobacco, chocolate, clove, cedar",Black Fruit,"blackberry, plum, black fruit, black cherry, d...",Earthy,"leather, smoke, earthy, mushroom, game, minerals"
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.3458,Oaky,"oak, vanilla, chocolate, tobacco, coffee, cedar",Black Fruit,"blackberry, plum, black fruit, dark fruit, cas...",Earthy,"leather, smoke, earthy, minerals, mushroom, tar"
4,이기갈 꽁드리유,E.Guigal Condrieu,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"apricot, peach, pear, stone fruit, melon, apple",Earthy,"honey, stone, minerals, ginger, smoke, flint",Oaky,"vanilla, oak, butter, caramel, butterscotch, c..."


In [78]:
shin_df.isnull().sum()

와인이름          0
영어이름          0
한줄소개          0
와인타입          0
나라, 와이너리      0
포도종           0
용량            0
곁들일 음식        5
당도            0
산도            0
바디            0
와인설명          0
수상          401
와이너리이름        0
와이너리설명        0
와이너리특징        0
도수          568
탄닌          570
아로마1_대분류    416
아로마1_소분류    416
아로마2_대분류    418
아로마2_소분류    418
아로마3_대분류    426
아로마3_소분류    426
dtype: int64

In [119]:
shin_df2 = shin_df.copy()
shin_df2.dropna(subset = ['도수'], inplace = True)

In [153]:
shin_df2.isnull().sum()

와인이름          0
영어이름          0
한줄소개          0
와인타입          0
나라, 와이너리      0
포도종           0
용량            0
곁들일 음식        1
당도            0
산도            0
바디            0
와인설명          0
수상           97
와이너리이름        0
와이너리설명        0
와이너리특징        0
도수            0
탄닌          138
아로마1_대분류     44
아로마1_소분류     44
아로마2_대분류     44
아로마2_소분류     44
아로마3_대분류     44
아로마3_소분류     44
dtype: int64

In [122]:
shin_df2['산도'].value_counts()

산도 3    302
산도 2     16
산도 4     10
산도 1      1
Name: 산도, dtype: int64

In [124]:
shin_df2['바디'].value_counts()

바디 4    140
바디 5     92
바디 3     78
바디 2     19
Name: 바디, dtype: int64

In [129]:
shin_df2[['산도','바디','당도','도수']].value_counts()

산도    바디    당도    도수  
산도 3  바디 4  당도 1  13.0    60
      바디 5  당도 1  14.0    51
      바디 3  당도 1  13.0    29
      바디 5  당도 1  13.0    23
      바디 4  당도 1  12.0    15
                          ..
            당도 2  9.0      1
                  11.0     1
                  14.0     1
            당도 3  14.0     1
산도 4  바디 5  당도 1  14.0     1
Length: 76, dtype: int64

In [121]:
np.sort(shin_df2['도수'].unique())

array([ 1.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16.,
       17., 20.])

### 아로마 대분류, 소분류 표준화

#### 와인설명 아로마 분류

In [145]:
aroma_text = ' '.join(shin_df['와인설명'].tolist())
aroma_text 

'밝게 빛나는 진한 적색을 띠고 있으며\xa0붉은 딸기 류의 풍부한 향과 스파이시한 노트가 느껴진다. 과일 아로마가 풍부하면서도 입 안에서 느껴지는 질감이 풀 바디한 스타일이다. 끝 맛에서 섬세하고 우아한 여운이 남아 좋은 균형감을 보여준다. 35년 된 포도나무에서 수확한 포도를 전통적인 방식으로 양조했으며 수확연도로부터 6~8년 정도 더 두고 숙성시켜 마실 수 있다. 비오니에 포도품종 특유의 살구, 복숭아 등의 신선한 과일 향과 향긋한 아카시아 꽃 향이 풍부한 아로마를 만든다. 프루티한 맛과 바디감이 느껴지며 우아한 여운을 남긴다. 25년된 포도나무에서 수확한 포도로 양조했으며 스테인리스 스틸 탱크에서 숙성한다. 붉은 딸기, 라스베리, 체리 등의 진한 향과 오크에서 배어나는 토스트 향, 볶은 커피 향, 바닐라 향이 감미롭다. 우아하게 다듬어진 탄닌의 질감과 과일 맛의 균형감이 뛰어나며 풀 바디하고 풍부한 맛을 낸다. 평균수령 35년의 포도밭에서 재배한 포도로 양조했으며, 36개월 간 오크에서 숙성시켰다.  잘 익은 자두와 블랙베리, 라즈베리 잼처럼 진하고 응축된 과일 향과 섬세한 향신료 향이 난다. 입 안에서 파워풀한 과일 맛과 함께 우아한 바닐라 향, 스모키한 향이 어우러진다. 단단한 탄닌의 맛이 남성적인 스타일을 나타내고 있으며 38개월 간의 새 오크통 숙성에서 배어나는 다채로운 향과 여운이 인상적이다. 꼬뜨 로띠에서 최고로 꼽히는 일곱 군데의 밭에서 수확한 포도만 사용해 양조했다. 살구, 복숭아, 시트러스 계열의 달콤하고 새콤한 과일 향이 주된 아로마를 이룬다. 약간의 꽃 향기와 함께 입 안에서 풍부한 포도 과즙 맛과 산뜻한 산미, 화려한 향이 어우러진 여운을 남긴다. 복숭아, 살구 등의 달콤하고 신선한 과일 향과 꽃 향기가 우아한 아로마를 만들어낸다. 입 안에서 풍부한 과즙 맛과 산뜻한 산미, 섬세한 여운이 느껴지며 비오니에 품종 특유의 화려함과 우아함이 조화를 이룬다. 9개월 간 프랑스산 새 오크통에서 숙성시키며 연간 2만 병 가량 극소량으로 출시한

In [146]:
from konlpy.tag import Okt

okt = Okt()
morph = okt.pos(aroma_text)
morph

[('밝게', 'Verb'),
 ('빛나는', 'Verb'),
 ('진한', 'Adjective'),
 ('적색', 'Noun'),
 ('을', 'Josa'),
 ('띠', 'Noun'),
 ('고', 'Josa'),
 ('있으며', 'Adjective'),
 ('\xa0', 'Foreign'),
 ('붉은', 'Adjective'),
 ('딸기', 'Noun'),
 ('류', 'Noun'),
 ('의', 'Josa'),
 ('풍부한', 'Adjective'),
 ('향', 'Noun'),
 ('과', 'Josa'),
 ('스파이', 'Noun'),
 ('시한', 'Noun'),
 ('노트', 'Noun'),
 ('가', 'Josa'),
 ('느껴진다', 'Verb'),
 ('.', 'Punctuation'),
 ('과일', 'Noun'),
 ('아로마', 'Noun'),
 ('가', 'Josa'),
 ('풍부하면서도', 'Adjective'),
 ('입', 'Noun'),
 ('안', 'Noun'),
 ('에서', 'Josa'),
 ('느껴지는', 'Verb'),
 ('질감', 'Noun'),
 ('이', 'Josa'),
 ('풀', 'Noun'),
 ('바디', 'Noun'),
 ('한', 'Josa'),
 ('스타일', 'Noun'),
 ('이다', 'Josa'),
 ('.', 'Punctuation'),
 ('끝', 'Noun'),
 ('맛', 'Noun'),
 ('에서', 'Josa'),
 ('섬세하고', 'Adjective'),
 ('우아한', 'Adjective'),
 ('여운', 'Noun'),
 ('이', 'Josa'),
 ('남아', 'Noun'),
 ('좋은', 'Adjective'),
 ('균형', 'Noun'),
 ('감', 'Noun'),
 ('을', 'Josa'),
 ('보여준다', 'Verb'),
 ('.', 'Punctuation'),
 ('35년', 'Number'),
 ('된', 'Verb'),
 ('포도나무', 'Noun')

In [149]:
aroma_noun = []
for word,tag in morph:
    if (tag == 'Noun') and len(word)>=1:
        aroma_noun.append(word)
        
aroma_noun

['적색',
 '띠',
 '딸기',
 '류',
 '향',
 '스파이',
 '시한',
 '노트',
 '과일',
 '아로마',
 '입',
 '안',
 '질감',
 '풀',
 '바디',
 '스타일',
 '끝',
 '맛',
 '여운',
 '남아',
 '균형',
 '감',
 '포도나무',
 '수확',
 '포도',
 '전통',
 '방식',
 '양조',
 '수확',
 '연도로',
 '정도',
 '더',
 '숙성',
 '수',
 '포도',
 '품종',
 '특유',
 '살구',
 '복숭아',
 '등',
 '과일',
 '향',
 '향',
 '아카시아',
 '꽃',
 '향',
 '아로마',
 '프루티',
 '맛',
 '바디',
 '감',
 '여운',
 '포도나무',
 '수확',
 '포도',
 '양조',
 '스테인리스',
 '스틸',
 '탱크',
 '딸기',
 '라스베리',
 '체리',
 '등',
 '향',
 '오크',
 '토스트',
 '향',
 '커피',
 '향',
 '바닐라',
 '향',
 '탄닌',
 '질감',
 '과일',
 '맛',
 '균형',
 '감',
 '풀',
 '바디',
 '맛',
 '평균',
 '수령',
 '포도',
 '밭',
 '재배',
 '포도',
 '양조',
 '개월',
 '간',
 '오크',
 '숙성',
 '자두',
 '블랙베리',
 '라즈베리',
 '잼',
 '응축',
 '과일',
 '향',
 '향신료',
 '향',
 '입',
 '안',
 '파워',
 '풀',
 '과일',
 '맛',
 '바닐라',
 '향',
 '스모키',
 '향',
 '탄닌',
 '맛',
 '남성',
 '스타일',
 '개월',
 '간의',
 '새',
 '오크',
 '통',
 '숙성',
 '채',
 '향',
 '여운',
 '인상',
 '꼬뜨',
 '로띠',
 '최고',
 '일곱',
 '군데',
 '밭',
 '수확',
 '포도',
 '사용',
 '양조',
 '살구',
 '복숭아',
 '트러스',
 '계열',
 '콤',
 '과일',
 '향',
 '아로마',
 '약간',
 '꽃',
 '향기',
 

In [152]:
from collections import Counter

one_dict = dict(Counter(aroma_noun).most_common())
one_dict

{'향': 1316,
 '와인': 976,
 '맛': 699,
 '과일': 677,
 '오크': 662,
 '포도': 648,
 '수': 476,
 '아로마': 446,
 '개월': 429,
 '여운': 424,
 '등': 366,
 '숙성': 328,
 '수확': 312,
 '탄닌': 304,
 '발효': 296,
 '배럴': 290,
 '띠': 289,
 '후': 288,
 '안': 286,
 '산미': 284,
 '양조': 282,
 '입': 270,
 '균형': 239,
 '체리': 218,
 '산': 218,
 '감': 217,
 '스타일': 209,
 '프랑스': 205,
 '더': 202,
 '재배': 201,
 '간': 200,
 '탱크': 194,
 '밭': 186,
 '스틸': 179,
 '색': 176,
 '바디': 175,
 '꽃': 173,
 '통': 173,
 '스테인리스': 172,
 '복합': 172,
 '병입': 168,
 '품종': 160,
 '미네랄': 148,
 '캐릭터': 147,
 '풀': 139,
 '출시': 135,
 '과정': 135,
 '사용': 125,
 '약간': 124,
 '자두': 122,
 '새': 122,
 '블랙': 122,
 '향신료': 118,
 '블랙베리': 114,
 '의': 114,
 '손': 112,
 '스파이': 105,
 '화이트': 105,
 '바닐라': 103,
 '루비': 102,
 '트러스': 101,
 '질감': 100,
 '채': 100,
 '병': 100,
 '생산': 96,
 '딸기': 95,
 '과실': 93,
 '해': 91,
 '노트': 89,
 '입안': 88,
 '레드': 88,
 '미가': 87,
 '특유': 85,
 '샴페인': 85,
 '시한': 84,
 '빈티': 84,
 '베리': 80,
 '향기': 79,
 '로': 79,
 '것': 79,
 '복숭아': 78,
 '효모': 77,
 '빛': 77,
 '초콜릿': 76,
 '과즙': 75,
 '가득': 7

In [155]:
# counter 된 flavor 골라내기

aroma_li = ['과일','오크','체리','스틸','꽃','스테인리스','미네랄','풀','자두','블랙베리','블랙커런트','바닐라','시트러스',
            '딸기','과실','베리','복숭아','효모','초콜릿','과즙','허브','라즈베리','커런트','레몬','감초','후추','사과','열대과일',
            '토양','다크','프루티','오렌지','크리미','자몽','실크','배','살구','벨벳','줄기','잎','스모키','민트','블루베리','파인애플',
            '볏짚','벌꿀','버터','장미','아몬드','커피','가죽','토스트','무화과','플로럴','단맛','멜론','제비꽃','라임','담뱃잎',
            '모카','코코아','흙내','토스티','시나몬','아카시아','헤이즐넛','꿀', '그린','발사믹','라스베리','리치','송이','콘크리트',
            '견과류','말보로','삼나무','애플','라벤더','바이올렛','올리브','빵','바나나','캐러멜','버섯','담배','망고','크랜베리','산딸기',
            '페퍼','부싯돌','세이지','코스모스','꽃잎','흑연','건포도','머스크','흙','장미꽃','석회질','시멘트','우유','트러플','로즈마리',
            '생강','자스민','숲','파파야','석류','우드','고목','호두','시럽','꽃향','프룻티','프룻','오일','이산화황','스테인레스',
            '청포도','곰팡이','레드커런트','버찌','에스프레소','육두구','타르','월계수','캐모마일','타르트','쥬이시','이슬','초콜렛',
            '페퍼민트','통후추','귤']

In [159]:
aroma_flavor = aroma_li.copy()
aroma_flavor

['과일',
 '오크',
 '체리',
 '스틸',
 '꽃',
 '스테인리스',
 '미네랄',
 '풀',
 '자두',
 '블랙베리',
 '블랙커런트',
 '바닐라',
 '시트러스',
 '딸기',
 '과실',
 '베리',
 '복숭아',
 '효모',
 '초콜릿',
 '과즙',
 '허브',
 '라즈베리',
 '커런트',
 '레몬',
 '감초',
 '후추',
 '사과',
 '열대과일',
 '토양',
 '다크',
 '프루티',
 '오렌지',
 '크리미',
 '자몽',
 '실크',
 '배',
 '살구',
 '벨벳',
 '줄기',
 '잎',
 '스모키',
 '민트',
 '블루베리',
 '파인애플',
 '볏짚',
 '벌꿀',
 '버터',
 '장미',
 '아몬드',
 '커피',
 '가죽',
 '토스트',
 '무화과',
 '플로럴',
 '단맛',
 '멜론',
 '제비꽃',
 '라임',
 '담뱃잎',
 '모카',
 '코코아',
 '흙내',
 '토스티',
 '시나몬',
 '아카시아',
 '헤이즐넛',
 '꿀',
 '그린',
 '발사믹',
 '라스베리',
 '리치',
 '송이',
 '콘크리트',
 '견과류',
 '말보로',
 '삼나무',
 '애플',
 '라벤더',
 '바이올렛',
 '올리브',
 '빵',
 '바나나',
 '캐러멜',
 '버섯',
 '담배',
 '망고',
 '크랜베리',
 '산딸기',
 '페퍼',
 '부싯돌',
 '세이지',
 '코스모스',
 '꽃잎',
 '흑연',
 '건포도',
 '머스크',
 '흙',
 '장미꽃',
 '석회질',
 '시멘트',
 '우유',
 '트러플',
 '로즈마리',
 '생강',
 '자스민',
 '숲',
 '파파야',
 '석류',
 '우드',
 '고목',
 '호두',
 '시럽',
 '꽃향',
 '프룻티',
 '프룻',
 '오일',
 '이산화황',
 '스테인레스',
 '청포도',
 '곰팡이',
 '레드커런트',
 '버찌',
 '에스프레소',
 '육두구',
 '타르',
 '월계수',
 '캐모마일',
 '타르트',
 '쥬이시',
 '이슬',
 '초콜렛',

In [162]:
# flavor3파일의 flavor 가져오기

flavor3 = '건포도,가죽, 고기, 타바코, 담배 풀, 숲, 카라멜, 커피, 토스트,휘발유, 꿀, 버섯, 석유, 부싯돌, 바닐라, 정향, 코코넛, 향나무, 담배, 초콜릿, 커피, 요거트, 버터, 크림, 치즈, 빵, 비스킷, 패스츄리, 빵반죽, 오렌지블러썸, 자스민, 아카시아, 라벤더, 장미, 사과, 배, 구즈베리, 포도, 청포도, 자몽, 레몬, 라임, 만다린, 오렌지, 매실, 복숭아, 살구, 앵두, 멜론, 구아바, 파인애플, 리치, 패션후르츠, 바나나, 망고, 레드커런트, 크랜베리, 라즈베리, 딸기, 레드체리, 레드자두, 블랙커런트, 블랙베리, 블랙체리, 블랙자두, 블랙체리, 청피망, 풀, 잔디, 토마토 잎, 토마토 리프, 아스파라거스, 담배, 건초, 유칼립투스, 민트, 딜, 펜넬, 후추, 감초'
flavor3 = flavor3.split(', ')
for f in flavor3:
    aroma_flavor.append(f)

aroma_flavor = list(set(aroma_flavor))
aroma_flavor

['정향',
 '감초',
 '망고',
 '흙',
 '벨벳',
 '스테인레스',
 '레몬',
 '프룻',
 '토스티',
 '자몽',
 '부싯돌',
 '블랙베리',
 '스테인리스',
 '통후추',
 '레드체리',
 '가죽',
 '체리',
 '말보로',
 '에스프레소',
 '배',
 '아몬드',
 '라즈베리',
 '애플',
 '자스민',
 '타르',
 '크림',
 '잎',
 '블랙체리',
 '레드커런트',
 '꽃향',
 '월계수',
 '미네랄',
 '견과류',
 '빵',
 '그린',
 '우유',
 '오렌지블러썸',
 '꽃잎',
 '삼나무',
 '이슬',
 '시럽',
 '숲',
 '헤이즐넛',
 '올리브',
 '오일',
 '크리미',
 '타바코',
 '버터',
 '후추',
 '살구',
 '효모',
 '앵두',
 '구아바',
 '건초',
 '단맛',
 '커런트',
 '베리',
 '토양',
 '시멘트',
 '빵반죽',
 '로즈마리',
 '줄기',
 '블랙커런트',
 '프루티',
 '멜론',
 '토스트',
 '크랜베리',
 '꽃',
 '라벤더',
 '담배',
 '과즙',
 '만다린',
 '건포도,가죽',
 '매실',
 '토마토 리프',
 '담배 풀',
 '시트러스',
 '장미',
 '트러플',
 '토마토 잎',
 '패스츄리',
 '모카',
 '포도',
 '캐모마일',
 '흙내',
 '프룻티',
 '과실',
 '송이',
 '제비꽃',
 '세이지',
 '레드자두',
 '석회질',
 '아카시아',
 '유칼립투스',
 '블랙자두',
 '발사믹',
 '과일',
 '석유',
 '민트',
 '파인애플',
 '열대과일',
 '허브',
 '장미꽃',
 '요거트',
 '건포도',
 '풀',
 '실크',
 '페퍼',
 '바나나',
 '펜넬',
 '육두구',
 '블루베리',
 '사과',
 '우드',
 '오렌지',
 '곰팡이',
 '향나무',
 '이산화황',
 '오크',
 '고목',
 '리치',
 '벌꿀',
 '초콜렛',
 '생강',
 '버섯',
 '흑연',
 '볏짚',
 '초콜릿',
 '산딸기

# 아로마 flavor

## 신세계 아로마 분류

### (수정)아로마 딕셔너리 만들기_0917

In [1]:
# 신세계+vivino raw 데이터프레임 재생성
import pandas as pd

wine_for_aroma = pd.read_pickle('shin_df_vivino.pkl')
wine_for_aroma

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,...,와이너리설명,와이너리특징,도수,탄닌,아로마1_대분류,아로마1_소분류,아로마2_대분류,아로마2_소분류,아로마3_대분류,아로마3_소분류
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",당도 1,산도 2,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,47.9624,Black Fruit,"plum, blackberry, dark fruit, black fruit, bla...",Oaky,"oak, vanilla, tobacco, chocolate, cedar, clove",Earthy,"earthy, leather, smoke, minerals, mushroom, stone"
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"peach, apricot, pear, apple, melon, stone fruit",Earthy,"minerals, honey, stone, earthy, smoke, ginger",Citrus,"citrus, lemon, grapefruit, lime, orange, tange..."
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.6574,Oaky,"oak, vanilla, tobacco, chocolate, clove, cedar",Black Fruit,"blackberry, plum, black fruit, black cherry, d...",Earthy,"leather, smoke, earthy, mushroom, game, minerals"
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.3458,Oaky,"oak, vanilla, chocolate, tobacco, coffee, cedar",Black Fruit,"blackberry, plum, black fruit, dark fruit, cas...",Earthy,"leather, smoke, earthy, minerals, mushroom, tar"
4,이기갈 꽁드리유,E.Guigal Condrieu,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"apricot, peach, pear, stone fruit, melon, apple",Earthy,"honey, stone, minerals, ginger, smoke, flint",Oaky,"vanilla, oak, butter, caramel, butterscotch, c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,까테나 자파타 까테나 알타 까베르네 소비뇽,Catena Zapata Catena Alta Cabernet Sauvignon,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,까베르네 소비뇽 100%,750ml,"치킨 커틀렛, 치즈 리조또, 숙성이 잘 된 치즈",당도 1,산도 3,...,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,NaN,42.9292,Black Fruit,"blackberry, plum, blackcurrant, black cherry, ...",Oaky,"oak, chocolate, mocha, caramel, vanilla, incense",Red Fruit,"raspberry, cherry, red fruit, strawberry, red ..."
893,까테나 자파타 까테나 알타 말벡,Catena Zapata Catena Alta Malbec,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말백 100%,750ml,"달콤한 육포, 스테이크, 훈연 소시지, 치즈",당도 1,산도 3,...,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,14.0,32.8264,Black Fruit,"blackberry, plum, blueberry, black fruit, dark...",Oaky,"oak, vanilla, chocolate, tobacco, dark chocola...",Earthy,"leather, earthy, smoke, minerals, cocoa, mushroom"
894,까테나 자파타 니카시아 말벡,Catena Zapata Nicasia Malbec,까테나 자파타 최고도 빈야드에서 생산되는 100% 말벡 와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,750ml,"양갈비, 로스트비프, 트러플 치즈 파니니, 머쉬룸 파스타",당도 1,산도 3,...,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,14.0,27.0405,Oaky,"oak, chocolate, vanilla, tobacco, coffee, dark...",Black Fruit,"blackberry, plum, black fruit, blueberry, cass...",Earthy,"leather, cocoa, earthy, mi

In [35]:
# 최종 정리한 flavor 파일 불러오기

flavor_final = pd.read_excel('C:/cakd7/cakd7_class/와인추천솔루션/flavor5.xlsx').loc[:15]
flavor_final

,발효과정,중분류,종류
0,3차(tertiary),말린 과일,건포도
1,3차(tertiary),자연(Earthy),"가죽, 타바코, 담배풀, 담배 풀, 카라멜, 커피, 토스트, 휘발유, 꿀, 오크 향..."
2,2차(secondary),오크(Oak),"바닐라, 정향, 코코넛, 향나무, 타르, 초콜릿, 커피, 초콜렛, 말보로, 육두구,..."
3,2차(secondary),유산발효,"요거트, 버터, 크림, 치즈, 우유, 크리미"
4,2차(secondary),이스트,"빵, 비스킷, 패스츄리, 빵반죽, 효모 향"
5,1차(primary),꽃,"오렌지블러썸, 자스민, 아카시아, 라벤더, 장미, 코스모스, 제비꽃, 바이올렛, 세이지"
6,1차(primary),녹색 과일,"사과, 배, 구즈베리, 애플"
7,1차(primary),시트러스,"자몽, 레몬, 라임, 만다린, 오렌지, 귤"
8,1차(primary),핵과,"매실, 복숭아, 살구, 앵두"
9,1차(primary),열대 과일,"멜론, 구아바, 파인애플, 리치, 패션후르츠, 바나나, 망고, 열대과일, 파파야, 무화과"


In [36]:
# 딕셔너리 형태로 변환

aroma_final = {}
for i in range(len(flavor_final)):
    value = flavor_final.iloc[i, 2].split(', ')
    aroma_final[flavor_final.iloc[i, 1]] = value

print(aroma_final)

# 모든 아로마

all_aroma = ', '.join(flavor_final['종류'].tolist()).split(', ')
all_aroma

{'말린 과일': ['건포도'], '자연(Earthy)': ['가죽', '타바코', '담배풀', '담배 풀', '카라멜', '커피', '토스트', '휘발유', '꿀', '오크 향', '버섯', '석유', '부싯돌', '미네랄', '토양', '흙', '흙내', '토스티', '송이', '담뱃잎', '벌꿀', '시멘트', '콘크리트', '흑연', '석회질'], '오크(Oak)': ['바닐라', '정향', '코코넛', '향나무', '타르', '초콜릿', '커피', '초콜렛', '말보로', '육두구', '우드', '에스프레소', '모카', '다크', '코코아', '아몬드', '호두', '견과류', '헤이즐넛', '캐러멜', '시럽'], '유산발효': ['요거트', '버터', '크림', '치즈', '우유', '크리미'], '이스트': ['빵', '비스킷', '패스츄리', '빵반죽', '효모 향'], '꽃': ['오렌지블러썸', '자스민', '아카시아', '라벤더', '장미', '코스모스', '제비꽃', '바이올렛', '세이지'], '녹색 과일': ['사과', '배', '구즈베리', '애플'], '시트러스': ['자몽', '레몬', '라임', '만다린', '오렌지', '귤'], '핵과': ['매실', '복숭아', '살구', '앵두'], '열대 과일': ['멜론', '구아바', '파인애플', '리치', '패션후르츠', '바나나', '망고', '열대과일', '파파야', '무화과'], '빨간 과일': ['레드커런트', '크랜베리', '라즈베리', '딸기', '레드체리', '레드자두', '체리', '자두', '베리', '라스베리', '산딸기', '버찌', '석류'], '까만 과일': ['블랙커런트', '블랙베리', '블랙체리', '블랙자두', '블랙체리', '블루베리'], '허베이셔스': ['청피망', '풀', '잔디', '토마토 잎', '토마토 리프', '아스파라거스 '], '말린 허브': ['담배', '건초', '월계수'], '허브': ['유칼립투스', '민트', '딜', '

['건포도',
 '가죽',
 '타바코',
 '담배풀',
 '담배 풀',
 '카라멜',
 '커피',
 '토스트',
 '휘발유',
 '꿀',
 '오크 향',
 '버섯',
 '석유',
 '부싯돌',
 '미네랄',
 '토양',
 '흙',
 '흙내',
 '토스티',
 '송이',
 '담뱃잎',
 '벌꿀',
 '시멘트',
 '콘크리트',
 '흑연',
 '석회질',
 '바닐라',
 '정향',
 '코코넛',
 '향나무',
 '타르',
 '초콜릿',
 '커피',
 '초콜렛',
 '말보로',
 '육두구',
 '우드',
 '에스프레소',
 '모카',
 '다크',
 '코코아',
 '아몬드',
 '호두',
 '견과류',
 '헤이즐넛',
 '캐러멜',
 '시럽',
 '요거트',
 '버터',
 '크림',
 '치즈',
 '우유',
 '크리미',
 '빵',
 '비스킷',
 '패스츄리',
 '빵반죽',
 '효모 향',
 '오렌지블러썸',
 '자스민',
 '아카시아',
 '라벤더',
 '장미',
 '코스모스',
 '제비꽃',
 '바이올렛',
 '세이지',
 '사과',
 '배',
 '구즈베리',
 '애플',
 '자몽',
 '레몬',
 '라임',
 '만다린',
 '오렌지',
 '귤',
 '매실',
 '복숭아',
 '살구',
 '앵두',
 '멜론',
 '구아바',
 '파인애플',
 '리치',
 '패션후르츠',
 '바나나',
 '망고',
 '열대과일',
 '파파야',
 '무화과',
 '레드커런트',
 '크랜베리',
 '라즈베리',
 '딸기',
 '레드체리',
 '레드자두',
 '체리',
 '자두',
 '베리',
 '라스베리',
 '산딸기',
 '버찌',
 '석류',
 '블랙커런트',
 '블랙베리',
 '블랙체리',
 '블랙자두',
 '블랙체리',
 '블루베리',
 '청피망',
 '풀',
 '잔디',
 '토마토 잎',
 '토마토 리프',
 '아스파라거스 ',
 '담배',
 '건초',
 '월계수',
 '유칼립투스',
 '민트',
 '딜',
 '펜넬',
 '허브',
 '페퍼민트',
 '로즈마리',
 '캐모마

In [59]:
wine_for_aroma['아로마 딕셔너리'] = np.nan


for idx in range(len(wine_for_aroma['와인설명'])):
    dict_temp = {}
    
    for key in aroma_final.keys():
        dict_temp[key] =[]
        
        for i in aroma_final[key]:
            word = wine_for_aroma.loc[idx, '와인설명']
            
            if word.find(i) != -1:
                dict_temp[key].append(i)
    
    dict_temp2 = {}
    for key, values in dict_temp.items():
        if len(dict_temp[key]) > 0:
            dict_temp2[key] = values
    print(len(dict_temp2.keys()))
    print(dict_temp2)
    
    wine_for_aroma.loc[idx, '아로마 딕셔너리'] = str(dict_temp2)
    

2
{'빨간 과일': ['딸기'], '허베이셔스': ['풀']}
2
{'꽃': ['아카시아'], '핵과': ['복숭아', '살구']}
5
{'자연(Earthy)': ['커피', '토스트'], '오크(Oak)': ['바닐라', '커피'], '녹색 과일': ['배'], '빨간 과일': ['딸기', '체리', '베리', '라스베리'], '허베이셔스': ['풀']}
5
{'오크(Oak)': ['바닐라'], '녹색 과일': ['배'], '빨간 과일': ['라즈베리', '자두', '베리'], '까만 과일': ['블랙베리'], '허베이셔스': ['풀']}
1
{'핵과': ['복숭아', '살구']}
1
{'핵과': ['복숭아', '살구']}
4
{'자연(Earthy)': ['오크 향'], '녹색 과일': ['배'], '빨간 과일': ['딸기', '체리'], '허베이셔스': ['풀']}
4
{'오크(Oak)': ['헤이즐넛'], '빨간 과일': ['자두', '베리', '라스베리'], '까만 과일': ['블랙베리'], '허베이셔스': ['풀']}
3
{'열대 과일': ['무화과'], '빨간 과일': ['자두'], '스파이스': ['후추', '감초', '통후추']}
3
{'자연(Earthy)': ['꿀', '오크 향', '미네랄'], '꽃': ['아카시아'], '허베이셔스': ['풀']}
3
{'자연(Earthy)': ['꿀'], '꽃': ['아카시아'], '허베이셔스': ['풀']}
2
{'허베이셔스': ['풀'], '스파이스': ['감초']}
5
{'오크(Oak)': ['바닐라'], '녹색 과일': ['배'], '빨간 과일': ['딸기', '체리'], '까만 과일': ['블랙커런트'], '스파이스': ['감초']}
4
{'자연(Earthy)': ['부싯돌', '미네랄'], '녹색 과일': ['배'], '빨간 과일': ['체리', '베리', '라스베리'], '까만 과일': ['블랙베리']}
4
{'자연(Earthy)': ['미네랄'], '빨간 과일': ['라즈베리', '자두',

2
{'녹색 과일': ['배'], '빨간 과일': ['체리']}
2
{'허브': ['허브'], '스파이스': ['후추']}
2
{'시트러스': ['자몽', '레몬'], '열대 과일': ['리치', '열대과일']}
1
{'녹색 과일': ['배']}
1
{'녹색 과일': ['배']}
5
{'자연(Earthy)': ['흙', '흙내'], '오크(Oak)': ['코코아'], '빨간 과일': ['체리', '베리'], '까만 과일': ['블랙베리', '블랙체리', '블랙체리'], '허베이셔스': ['풀']}
5
{'자연(Earthy)': ['가죽', '시멘트'], '오크(Oak)': ['바닐라', '초콜릿'], '녹색 과일': ['배'], '빨간 과일': ['딸기', '체리'], '스파이스': ['후추']}
4
{'오크(Oak)': ['다크'], '빨간 과일': ['체리', '베리', '버찌'], '허브': ['민트'], '스파이스': ['후추']}
2
{'유산발효': ['크리미'], '빨간 과일': ['레드커런트', '체리', '베리', '라스베리']}
4
{'자연(Earthy)': ['미네랄'], '꽃': ['장미'], '빨간 과일': ['라즈베리', '딸기', '베리'], '허브': ['잎']}
2
{'자연(Earthy)': ['미네랄'], '열대 과일': ['열대과일']}
2
{'자연(Earthy)': ['버섯', '흙', '흙내'], '빨간 과일': ['딸기', '체리', '자두', '베리', '라스베리']}
2
{'빨간 과일': ['딸기'], '허브': ['허브']}
1
{'자연(Earthy)': ['미네랄']}
1
{'허베이셔스': ['풀']}
4
{'자연(Earthy)': ['오크 향'], '빨간 과일': ['체리', '베리'], '까만 과일': ['블랙베리', '블랙체리', '블랙체리'], '허베이셔스': ['풀']}
2
{'빨간 과일': ['체리', '자두', '베리'], '까만 과일': ['블랙베리', '블랙체리', '블랙체리']}
4
{'이스트': 

1
{'녹색 과일': ['배']}
3
{'꽃': ['장미'], '녹색 과일': ['배'], '시트러스': ['오렌지']}
3
{'녹색 과일': ['배', '애플'], '핵과': ['살구'], '열대 과일': ['파인애플']}
1
{'빨간 과일': ['딸기']}
3
{'녹색 과일': ['배'], '빨간 과일': ['체리'], '까만 과일': ['블랙체리', '블랙체리']}
5
{'자연(Earthy)': ['오크 향', '토양'], '녹색 과일': ['배'], '빨간 과일': ['라즈베리', '자두', '베리'], '까만 과일': ['블랙커런트'], '허베이셔스': ['풀']}
5
{'오크(Oak)': ['초콜릿', '모카'], '녹색 과일': ['배'], '빨간 과일': ['레드커런트', '체리', '베리'], '까만 과일': ['블랙베리', '블랙체리', '블랙체리'], '허베이셔스': ['풀']}
7
{'자연(Earthy)': ['커피'], '오크(Oak)': ['커피', '코코아'], '녹색 과일': ['배'], '빨간 과일': ['체리', '자두', '베리'], '까만 과일': ['블루베리'], '허베이셔스': ['풀'], '스파이스': ['후추']}
4
{'자연(Earthy)': ['미네랄'], '녹색 과일': ['배', '애플'], '시트러스': ['오렌지'], '열대 과일': ['파인애플']}
6
{'자연(Earthy)': ['담뱃잎'], '오크(Oak)': ['바닐라', '초콜릿'], '녹색 과일': ['배'], '빨간 과일': ['라즈베리', '자두', '베리'], '허베이셔스': ['풀'], '허브': ['잎']}
5
{'오크(Oak)': ['초콜릿', '모카', '코코아'], '녹색 과일': ['배'], '빨간 과일': ['레드커런트', '체리', '베리'], '까만 과일': ['블랙베리', '블랙체리', '블랙체리'], '스파이스': ['후추']}
2
{'녹색 과일': ['배'], '빨간 과일': ['라즈베리', '딸기', '체리', '자두

6
{'자연(Earthy)': ['미네랄', '토양', '석회질'], '꽃': ['제비꽃'], '녹색 과일': ['배'], '빨간 과일': ['체리', '베리'], '까만 과일': ['블랙베리', '블랙체리', '블랙체리'], '허베이셔스': ['풀']}
6
{'자연(Earthy)': ['흙', '흙내'], '오크(Oak)': ['타르'], '빨간 과일': ['크랜베리', '베리'], '까만 과일': ['블랙베리'], '말린 허브': ['월계수'], '허브': ['허브', '잎']}
5
{'꽃': ['라벤더'], '녹색 과일': ['배'], '빨간 과일': ['베리'], '까만 과일': ['블랙베리', '블루베리'], '스파이스': ['감초']}
4
{'자연(Earthy)': ['토스티'], '녹색 과일': ['사과', '배'], '핵과': ['살구'], '열대 과일': ['멜론', '바나나']}
3
{'오크(Oak)': ['초콜릿'], '녹색 과일': ['사과', '배'], '허브': ['허브']}
7
{'자연(Earthy)': ['송이'], '유산발효': ['크리미'], '꽃': ['장미'], '녹색 과일': ['배'], '빨간 과일': ['라즈베리', '체리', '베리'], '허브': ['잎'], '스파이스': ['후추', '통후추']}
3
{'자연(Earthy)': ['오크 향'], '녹색 과일': ['배'], '허베이셔스': ['풀']}
4
{'자연(Earthy)': ['미네랄'], '유산발효': ['크리미'], '시트러스': ['자몽'], '스파이스': ['후추']}
2
{'오크(Oak)': ['바닐라'], '빨간 과일': ['자두']}
6
{'자연(Earthy)': ['담뱃잎'], '오크(Oak)': ['바닐라', '정향'], '녹색 과일': ['배'], '빨간 과일': ['체리', '자두'], '까만 과일': ['블랙체리', '블랙체리'], '허브': ['잎']}
5
{'자연(Earthy)': ['토스티'], '오크(Oak)': ['바닐라'], 

8
{'자연(Earthy)': ['미네랄'], '오크(Oak)': ['초콜릿'], '녹색 과일': ['배'], '빨간 과일': ['레드커런트', '베리'], '까만 과일': ['블랙베리'], '허베이셔스': ['풀'], '허브': ['잎', '줄기'], '스파이스': ['감초']}
4
{'녹색 과일': ['배'], '빨간 과일': ['베리'], '까만 과일': ['블랙커런트', '블랙베리'], '스파이스': ['감초']}
4
{'자연(Earthy)': ['미네랄'], '녹색 과일': ['배'], '빨간 과일': ['레드커런트', '베리'], '까만 과일': ['블랙베리']}
2
{'유산발효': ['버터'], '스파이스': ['감초']}
3
{'녹색 과일': ['애플'], '시트러스': ['오렌지'], '열대 과일': ['파인애플', '바나나', '열대과일']}
3
{'자연(Earthy)': ['미네랄'], '오크(Oak)': ['코코아'], '이스트': ['빵']}
1
{'오크(Oak)': ['견과류']}
4
{'시트러스': ['자몽', '오렌지'], '빨간 과일': ['딸기'], '허브': ['민트', '페퍼민트'], '스파이스': ['페퍼']}
4
{'시트러스': ['오렌지'], '빨간 과일': ['딸기', '체리', '베리'], '허베이셔스': ['풀'], '스파이스': ['페퍼']}
2
{'자연(Earthy)': ['오크 향'], '녹색 과일': ['배']}
3
{'녹색 과일': ['배'], '빨간 과일': ['딸기', '자두'], '허베이셔스': ['풀']}
5
{'자연(Earthy)': ['꿀'], '오크(Oak)': ['초콜릿'], '녹색 과일': ['배'], '빨간 과일': ['체리', '자두', '베리'], '까만 과일': ['블랙베리', '블랙체리', '블랙체리']}
2
{'유산발효': ['버터'], '녹색 과일': ['배']}
4
{'자연(Earthy)': ['미네랄'], '오크(Oak)': ['초콜릿', '다크'], '시트러스': ['레몬

5
{'자연(Earthy)': ['미네랄'], '녹색 과일': ['배', '애플'], '시트러스': ['자몽'], '열대 과일': ['파인애플'], '허브': ['허브']}
2
{'자연(Earthy)': ['부싯돌', '송이'], '오크(Oak)': ['말보로', '헤이즐넛']}
3
{'녹색 과일': ['애플'], '시트러스': ['레몬'], '열대 과일': ['파인애플']}
3
{'자연(Earthy)': ['송이', '석회질'], '꽃': ['장미'], '녹색 과일': ['배']}
3
{'자연(Earthy)': ['토양'], '오크(Oak)': ['아몬드', '시럽'], '녹색 과일': ['배']}
1
{'자연(Earthy)': ['토양', '석회질']}
1
{'자연(Earthy)': ['토양']}
0
{}
1
{'시트러스': ['레몬']}
4
{'자연(Earthy)': ['미네랄'], '오크(Oak)': ['바닐라'], '유산발효': ['크리미'], '시트러스': ['레몬', '오렌지']}
5
{'유산발효': ['크리미'], '녹색 과일': ['사과'], '시트러스': ['레몬'], '열대 과일': ['리치'], '허베이셔스': ['풀']}
3
{'자연(Earthy)': ['미네랄'], '녹색 과일': ['애플'], '시트러스': ['레몬']}
5
{'자연(Earthy)': ['미네랄'], '시트러스': ['레몬'], '핵과': ['살구'], '빨간 과일': ['베리'], '허베이셔스': ['풀']}
5
{'오크(Oak)': ['바닐라'], '녹색 과일': ['배'], '열대 과일': ['무화과'], '까만 과일': ['블랙커런트'], '스파이스': ['후추']}
3
{'오크(Oak)': ['바닐라', '초콜릿', '다크'], '빨간 과일': ['베리'], '까만 과일': ['블루베리']}
4
{'자연(Earthy)': ['타바코'], '빨간 과일': ['베리'], '까만 과일': ['블루베리'], '스파이스': ['페퍼']}
3
{'오크(Oak)': ['

3
{'녹색 과일': ['배'], '빨간 과일': ['라즈베리', '베리'], '까만 과일': ['블랙베리']}
4
{'오크(Oak)': ['다크'], '빨간 과일': ['베리'], '허브': ['유칼립투스'], '스파이스': ['후추']}


In [60]:
wine_for_aroma.head(2)

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,...,와이너리특징,도수,탄닌,아로마1_대분류,아로마1_소분류,아로마2_대분류,아로마2_소분류,아로마3_대분류,아로마3_소분류,아로마 딕셔너리
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",1,2,...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,1.0,Black Fruit,"plum, blackberry, dark fruit, black fruit, bla...",Oaky,"oak, vanilla, tobacco, chocolate, cedar, clove",Earthy,"earthy, leather, smoke, minerals, mushroom, stone","{'빨간 과일': ['딸기'], '허베이셔스': ['풀']}"
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",1,3,...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"peach, apricot, pear, apple, melon, stone fruit",Earthy,"minerals, honey, stone, earthy, smoke, ginger",Citrus,"citrus, lemon, grapefruit, lime, orange, tange...","{'꽃': ['아카시아'], '핵과': ['복숭아', '살구']}"


## vivino 아로마 분류(미사용)

In [70]:
set(shin_df['아로마1_대분류'].tolist()), set(shin_df['아로마2_대분류'].tolist()), set(shin_df['아로마3_대분류'].tolist())

({'',
  'Ageing',
  'Black Fruit',
  'Citrus',
  'Earthy',
  'Oaky',
  'Red Fruit',
  'Tree Fruit',
  'Tropical',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  na

In [76]:
aroma = list(set([
'Ageing',
  'Black Fruit',
  'Citrus',
  'Earthy',
  'Oaky',
  'Red Fruit',
  'Tree Fruit',
  'Tropical',
'Ageing',
  'Black Fruit',
  'Citrus',
  'Earthy',
  'Microbio',
  'Oaky',
  'Red Fruit',
  'Spices',
  'Tree Fruit',
  'Tropical',
'Ageing',
  'Black Fruit',
  'Citrus',
  'Dried Fruit',
  'Earthy',
  'Floral',
  'Microbio',
  'Oaky',
  'Red Fruit',
  'Spices',
  'Tree Fruit',
  'Tropical',
  'Vegetal'
]))
aroma

['Vegetal',
 'Tropical',
 'Ageing',
 'Microbio',
 'Earthy',
 'Black Fruit',
 'Citrus',
 'Dried Fruit',
 'Red Fruit',
 'Floral',
 'Tree Fruit',
 'Oaky',
 'Spices']

In [103]:
for i in list(set(', '.join(shin_df.loc[shin_df[f'아로마1_대분류'] == 'Tropical', f'아로마1_소분류'].tolist()).split(', '))):
    aroma_dict['Tropical'].append(i)

In [100]:
aroma_dict = {}
aroma_dict['Tropical'] = []

In [101]:
aroma_dict['Tropical'].append('a')

In [113]:
aroma_dict = {}

for a in aroma:
    aroma_dict[a] = []
    
for a in aroma:
    for i in range(1,4):
        for j in list(set(', '.join(shin_df.loc[shin_df[f'아로마{i}_대분류'] == a, f'아로마{i}_소분류'].tolist()).split(', '))):
            if j != '':
                aroma_dict[a].append(j)
                
for a in aroma:
    aroma_dict[a] = list(set(aroma_dict[a]))
    print(len(aroma_dict[a]))
    
aroma_dict

8
9
12
9
46
20
20
6
21
6
19
35
23


{'Vegetal': ['tomato leaf',
  'rhubarb',
  'hay',
  'gooseberry',
  'straw',
  'grass',
  'asparagus',
  'tomato'],
 'Tropical': ['lychee',
  'fruit cup',
  'mango',
  'tropical',
  'pineapple',
  'passion fruit',
  'kiwi',
  'papaya',
  'guava'],
 'Ageing': ['almond',
  'marzipan',
  'toasted almond',
  'nutty',
  'biscuit',
  'dried fig',
  'walnut',
  'roasted almond',
  'toast',
  'brioche',
  'walnut oil',
  'hazelnut'],
 'Microbio': ['oil',
  'yogurt',
  'cream',
  'parmesan cheese',
  'banana',
  'cheese',
  'creme fraiche',
  'yeast',
  'lemon curd'],
 'Earthy': ['roasted meat',
  'grilled meat',
  'minerals',
  'mushroom',
  'saline',
  'slate',
  'balsamic',
  'ginger',
  'potting soil',
  'plastic',
  'honey',
  'graphite',
  'petroleum',
  'barbecue meat',
  'beeswax',
  'stone',
  'forest floor',
  'caramel popcorn',
  'crushed gravel',
  'chalk',
  'seashell',
  'kerosene',
  'cured meat',
  'iron',
  'honeycomb',
  'salt',
  'leather',
  'ash',
  'exotic spice',
  'game'

In [18]:
aroma_df = pd.DataFrame(columns = ['중분류','소분류'])
aroma_df['중분류'] = aroma_dict.keys()
aroma_df['소분류'] = aroma_dict.values()
aroma_df

NameError: name 'aroma_dict' is not defined

In [139]:
aroma_df.to_pickle('aroma_df_vivino.pkl')

## 데이터 전처리

In [43]:
wine_for_aroma

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,...,와이너리설명,와이너리특징,도수,탄닌,아로마1_대분류,아로마1_소분류,아로마2_대분류,아로마2_소분류,아로마3_대분류,아로마3_소분류
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",당도 1,산도 2,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,47.9624,Black Fruit,"plum, blackberry, dark fruit, black fruit, bla...",Oaky,"oak, vanilla, tobacco, chocolate, cedar, clove",Earthy,"earthy, leather, smoke, minerals, mushroom, stone"
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"peach, apricot, pear, apple, melon, stone fruit",Earthy,"minerals, honey, stone, earthy, smoke, ginger",Citrus,"citrus, lemon, grapefruit, lime, orange, tange..."
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.6574,Oaky,"oak, vanilla, tobacco, chocolate, clove, cedar",Black Fruit,"blackberry, plum, black fruit, black cherry, d...",Earthy,"leather, smoke, earthy, mushroom, game, minerals"
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",750ml,"붉은 육류요리, 치즈",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,66.3458,Oaky,"oak, vanilla, chocolate, tobacco, coffee, cedar",Black Fruit,"blackberry, plum, black fruit, dark fruit, cas...",Earthy,"leather, smoke, earthy, minerals, mushroom, tar"
4,이기갈 꽁드리유,E.Guigal Condrieu,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",당도 1,산도 3,...,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"apricot, peach, pear, stone fruit, melon, apple",Earthy,"honey, stone, minerals, ginger, smoke, flint",Oaky,"vanilla, oak, butter, caramel, butterscotch, c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,까테나 자파타 까테나 알타 까베르네 소비뇽,Catena Zapata Catena Alta Cabernet Sauvignon,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,까베르네 소비뇽 100%,750ml,"치킨 커틀렛, 치즈 리조또, 숙성이 잘 된 치즈",당도 1,산도 3,...,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,NaN,42.9292,Black Fruit,"blackberry, plum, blackcurrant, black cherry, ...",Oaky,"oak, chocolate, mocha, caramel, vanilla, incense",Red Fruit,"raspberry, cherry, red fruit, strawberry, red ..."
893,까테나 자파타 까테나 알타 말벡,Catena Zapata Catena Alta Malbec,최적의 시기에 수확한 포도를 선별해 만든 아르헨티나 레드와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말백 100%,750ml,"달콤한 육포, 스테이크, 훈연 소시지, 치즈",당도 1,산도 3,...,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,14.0,32.8264,Black Fruit,"blackberry, plum, blueberry, black fruit, dark...",Oaky,"oak, vanilla, chocolate, tobacco, dark chocola...",Earthy,"leather, earthy, smoke, minerals, cocoa, mushroom"
894,까테나 자파타 니카시아 말벡,Catena Zapata Nicasia Malbec,까테나 자파타 최고도 빈야드에서 생산되는 100% 말벡 와인,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,750ml,"양갈비, 로스트비프, 트러플 치즈 파니니, 머쉬룸 파스타",당도 1,산도 3,...,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,14.0,27.0405,Oaky,"oak, chocolate, vanilla, tobacco, coffee, dark...",Black Fruit,"blackberry, plum, black fruit, blueberry, cass...",Earthy,"leather, cocoa, earthy, mi

#### 당도, 산도, 바디 숫자 데이터 추출

In [61]:
import re

wine_for_aroma['당도'] = wine_for_aroma['당도'].apply(lambda x: x[-1])
wine_for_aroma['산도'] = wine_for_aroma['산도'].apply(lambda x: x[-1])
wine_for_aroma['바디'] = wine_for_aroma['바디'].apply(lambda x: x[-1])

In [62]:
wine_for_aroma.head()

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,...,와이너리특징,도수,탄닌,아로마1_대분류,아로마1_소분류,아로마2_대분류,아로마2_소분류,아로마3_대분류,아로마3_소분류,아로마 딕셔너리
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",1,2,...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,1.0,Black Fruit,"plum, blackberry, dark fruit, black fruit, bla...",Oaky,"oak, vanilla, tobacco, chocolate, cedar, clove",Earthy,"earthy, leather, smoke, minerals, mushroom, stone","{'빨간 과일': ['딸기'], '허베이셔스': ['풀']}"
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",1,3,...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"peach, apricot, pear, apple, melon, stone fruit",Earthy,"minerals, honey, stone, earthy, smoke, ginger",Citrus,"citrus, lemon, grapefruit, lime, orange, tange...","{'꽃': ['아카시아'], '핵과': ['복숭아', '살구']}"
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",750ml,"붉은 육류요리, 치즈",1,3,...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,1.0,Oaky,"oak, vanilla, tobacco, chocolate, clove, cedar",Black Fruit,"blackberry, plum, black fruit, black cherry, d...",Earthy,"leather, smoke, earthy, mushroom, game, minerals","{'자연(Earthy)': ['커피', '토스트'], '오크(Oak)': ['바닐라..."
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",750ml,"붉은 육류요리, 치즈",1,3,...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,1.0,Oaky,"oak, vanilla, chocolate, tobacco, coffee, cedar",Black Fruit,"blackberry, plum, black fruit, dark fruit, cas...",Earthy,"leather, smoke, earthy, minerals, mushroom, tar","{'오크(Oak)': ['바닐라'], '녹색 과일': ['배'], '빨간 과일': ..."
4,이기갈 꽁드리유,E.Guigal Condrieu,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,750ml,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",1,3,...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"apricot, peach, pear, stone fruit, melon, apple",Earthy,"honey, stone, minerals, ginger, smoke, flint",Oaky,"vanilla, oak, butter, caramel, butterscotch, c...","{'핵과': ['복숭아', '살구']}"


#### 타닌 표준화

In [63]:
wine_for_aroma['당도'].value_counts()

1    758
2     55
4     35
5     34
3     13
       2
Name: 당도, dtype: int64

In [64]:
# 문자 -> 숫자
wine_for_aroma['탄닌'] = wine_for_aroma['탄닌'].astype(float)

# 범주화
def tan_cate(x):
    y = x
    if x<20: y = 1
    elif x<40: y =2
    elif x<60: y =3
    elif x<80: y =4
    elif x<=100: y =5
    return y
    
wine_for_aroma['탄닌'] = wine_for_aroma['탄닌'].apply(lambda x: tan_cate(x))
wine_for_aroma.head(2)

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,...,와이너리특징,도수,탄닌,아로마1_대분류,아로마1_소분류,아로마2_대분류,아로마2_소분류,아로마3_대분류,아로마3_소분류,아로마 딕셔너리
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",750ml,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",1,2,...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,1.0,Black Fruit,"plum, blackberry, dark fruit, black fruit, bla...",Oaky,"oak, vanilla, tobacco, chocolate, cedar, clove",Earthy,"earthy, leather, smoke, minerals, mushroom, stone","{'빨간 과일': ['딸기'], '허베이셔스': ['풀']}"
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,"""King of the Rhone""으로 불리는 E.Guigal의 와인입니다.",화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",750ml,"해산물이나 매콤한 요리, 간단한 핑거푸드",1,3,...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...",13.0,NaN,Tree Fruit,"peach, apricot, pear, apple, melon, stone fruit",Earthy,"minerals, honey, stone, earthy, smoke, ginger",Citrus,"citrus, lemon, grapefruit, lime, orange, tange...","{'꽃': ['아카시아'], '핵과': ['복숭아', '살구']}"


#### 와인 타입 표준화
-  레드 와인, 화이트 와인, 스파클링 와인(샴페인, 로제, 레드 스파클링), 로제 와인, 디저트 와인(와인칵테일, 스위트 와인)

In [65]:
wine_for_aroma['와인타입'].unique()

array(['레드 와인', '화이트 와인', '스파클링', '로제 와인', '디저트 와인'], dtype=object)

In [66]:
wine_for_aroma[wine_for_aroma['와인타입']=='포티파이드 와인']

,와인이름,영어이름,한줄소개,와인타입,"나라, 와이너리",포도종,용량,곁들일 음식,당도,산도,...,와이너리특징,도수,탄닌,아로마1_대분류,아로마1_소분류,아로마2_대분류,아로마2_소분류,아로마3_대분류,아로마3_소분류,아로마 딕셔너리


In [67]:
wine_for_aroma['와인타입'] = wine_for_aroma['와인타입'].replace(['화이트 스파클링 와인','샴페인','로제 샴페인', '로제 스파클링', '레드 스파클링 와인'],'스파클링')
wine_for_aroma['와인타입'] = wine_for_aroma['와인타입'].replace(['스위트 와인','와인칵테일'], '디저트 와인')
wine_for_aroma['와인타입'] = wine_for_aroma['와인타입'].replace(['포티파이드 와인'],'레드 와인')
wine_for_aroma['와인타입'].unique()

array(['레드 와인', '화이트 와인', '스파클링', '로제 와인', '디저트 와인'], dtype=object)

#### 포도종 확인

In [22]:
shin_df['포도종'].unique()

array(['시라 49%, 그르나쉬 48%, 무르베드르 3%',
       '비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%', '시라 96%, 비오니에 4%',
       '시라 93%, 비오니에 7%', '비오니에 100%', '시라 100%',
       '그르나쉬 80%, 시라 10%, 무르베드르 5%, 기타 5%', '마르산느 95%, 루산느 5%',
       '마르산느 90%, 루산느 10%', '그르나쉬 65%, 무르베드르 25%, 시라 10%', '말벡 100%',
       '말벡 27%, 메를로 20%, 까베르네 소비뇽 24%, 까베르네 프랑 25%, 쁘띠 베르도 4%',
       '모스카토 100%', '브라케토 100%', '바르베라 100%',
       '바르베라, 까베르네 소비뇽, 네비올로 (모두 같은 비율)', '샤르도네 100%', '네비올로 100%',
       '프레이사 100%', '템프라니요 85%, 그라시아노 15%',
       '리슬링 50%, 잼므링 20%, 리바너 20%, 겔버 무스카텔러 10%', '그뤼너 벨트리너 100%',
       '산지오베제 90%, 까나이올로 5%, 칠레지올로 5%',
       '프리미티보 40%(뿔리아), 몬테풀치아노 30%(아브루쪼), 네로다볼라 20%(시칠리아), 메를로 10%(베네토)',
       '비오니에 40%(시칠리아), 샤르도네 30%(트렌티노), 베르멘티노 20%(마렘마), 페코리노 10%(마르케)',
       '산지오베제 85%, 메를로 15%', '산지오베제 90%, 까베르네 소비뇽 10%', '브루넬로 100%',
       '산지오베제 60%, 까베르네 소비뇽 20%, 메를로 20%', '프로세코 100%', '네로 다볼라 100%',
       '피노누아 100%', '소비뇽 블랑 100%', '템프라니요 100%', '템프라니요 97%, 그라시아노 3%',
       '까베르네 소비뇽 1

In [23]:
set(re.findall('(\w+)',', '.join(shin_df['포도종'].unique().tolist())))

{'1',
 '10',
 '100',
 '11',
 '12',
 '13',
 '13종',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '23',
 '24',
 '25',
 '27',
 '28',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '37',
 '38',
 '4',
 '40',
 '43',
 '45',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '53',
 '55',
 '57',
 '6',
 '60',
 '61',
 '62',
 '63',
 '65',
 '66',
 '67',
 '7',
 '70',
 '72',
 '75',
 '77',
 '8',
 '80',
 '81',
 '83',
 '84',
 '85',
 '86',
 '88',
 '89',
 '9',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 'A',
 'AOC',
 'Fino',
 'Palomino',
 'Pedro',
 'Trepat',
 'Ximenez',
 '가르가네가',
 '가르나차',
 '같은',
 '갸메',
 '게부르츠트라미너',
 '겔버',
 '계열',
 '고우비에오',
 '그라시아노',
 '그라우부르군더',
 '그랑',
 '그레카니코',
 '그뤼너',
 '그르나쉬',
 '그르나슈',
 '그리',
 '그리나슈',
 '그리지오',
 '그린',
 '그릴로',
 '글레라',
 '기타',
 '기타품종',
 '기타품종10',
 '까나이올로',
 '까르메네르',
 '까르미네르',
 '까리깐테',
 '까리냐노',
 '까리냥',
 '까베르네',
 '까베르네소비뇽30',
 '까스텔라옹',
 '깔라독',
 '꼬르비나',
 '꼬르비노네',
 '꼴로리노',
 '꾸누아즈',
 '끌레레뜨',
 '나시오날',
 '네그로',
 '네그로아마로',
 '네라',
 '네렐로',
 '네로',
 '네

#### 컬럼순서 정렬

In [57]:
wine_for_aroma.columns

Index(['와인이름', '영어이름', '한줄소개', '와인타입', '나라, 와이너리', '포도종', '용량', '곁들일 음식', '당도',
       '산도', '바디', '와인설명', '수상', '와이너리이름', '와이너리설명', '와이너리특징', '도수', '탄닌',
       '아로마1_대분류', '아로마1_소분류', '아로마2_대분류', '아로마2_소분류', '아로마3_대분류', '아로마3_소분류'],
      dtype='object')

In [68]:
wine_df = wine_for_aroma[['와인이름', '영어이름', '와인타입', '나라, 와이너리', '포도종','도수','당도','산도','바디','탄닌',
                   '와이너리이름','와이너리설명','와이너리특징','아로마 딕셔너리']]
wine_df

,와인이름,영어이름,와인타입,"나라, 와이너리",포도종,도수,당도,산도,바디,탄닌,와이너리이름,와이너리설명,와이너리특징,아로마 딕셔너리
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",13.0,1,2,4,1.0,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...","{'빨간 과일': ['딸기'], '허베이셔스': ['풀']}"
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",13.0,1,3,3,NaN,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...","{'꽃': ['아카시아'], '핵과': ['복숭아', '살구']}"
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",13.0,1,3,5,1.0,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...","{'자연(Earthy)': ['커피', '토스트'], '오크(Oak)': ['바닐라..."
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",13.0,1,3,5,1.0,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...","{'오크(Oak)': ['바닐라'], '녹색 과일': ['배'], '빨간 과일': ..."
4,이기갈 꽁드리유,E.Guigal Condrieu,화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,13.0,1,3,4,NaN,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...","{'핵과': ['복숭아', '살구']}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,까테나 자파타 까테나 알타 까베르네 소비뇽,Catena Zapata Catena Alta Cabernet Sauvignon,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,까베르네 소비뇽 100%,NaN,1,3,5,1.0,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,"{'자연(Earthy)': ['토스트'], '오크(Oak)': ['바닐라'], '빨..."
893,까테나 자파타 까테나 알타 말벡,Catena Zapata Catena Alta Malbec,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말백 100%,14.0,1,3,4,1.0,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,"{'오크(Oak)': ['바닐라'], '빨간 과일': ['베리'], '까만 과일':..."
894,까테나 자파타 니카시아 말벡,Catena Zapata Nicasia Malbec,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,14.0,1,3,5,1.0,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,"{'오크(Oak)': ['초콜릿'], '꽃': ['바이올렛'], '빨간 과일': [..."
895,까테나 자파타 아르헨티노 말벡,Catena Zapata Argentino Malbec,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,14.0,1,4,5,1.0,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,"{'녹색 과일': ['배'], '빨간 과일': ['라즈베리', '베리'], '까만 ..."


In [69]:
wine_df['아로마 딕셔너리']

0                      {'빨간 과일': ['딸기'], '허베이셔스': ['풀']}
1                   {'꽃': ['아카시아'], '핵과': ['복숭아', '살구']}
2      {'자연(Earthy)': ['커피', '토스트'], '오크(Oak)': ['바닐라...
3      {'오크(Oak)': ['바닐라'], '녹색 과일': ['배'], '빨간 과일': ...
4                                  {'핵과': ['복숭아', '살구']}
                             ...                        
892    {'자연(Earthy)': ['토스트'], '오크(Oak)': ['바닐라'], '빨...
893    {'오크(Oak)': ['바닐라'], '빨간 과일': ['베리'], '까만 과일':...
894    {'오크(Oak)': ['초콜릿'], '꽃': ['바이올렛'], '빨간 과일': [...
895    {'녹색 과일': ['배'], '빨간 과일': ['라즈베리', '베리'], '까만 ...
896    {'오크(Oak)': ['다크'], '빨간 과일': ['베리'], '허브': ['유...
Name: 아로마 딕셔너리, Length: 897, dtype: object

#### 곁들일 음식 컬럼 합치기

In [70]:
dish_df = pd.read_pickle('foods_after.pkl')
dish_df.columns

Index(['와인이름', '영어이름', '한줄소개', '와인타입', '나라, 와이너리', '포도종', '용량', '곁들일 음식', '당도',
       '산도', '바디', '와인설명', '수상', '와이너리이름', '와이너리설명', '와이너리특징', '도수', '탄닌',
       '아로마1_대분류', '아로마1_소분류', '아로마2_대분류', '아로마2_소분류', '아로마3_대분류', '아로마3_소분류',
       '음식'],
      dtype='object')

In [71]:
dish_df[['곁들일 음식','음식']]

,곁들일 음식,음식
0,"차가운 육류요리나 가금류, 붉은육류요리, 치즈",육류 에피타이저
1,"해산물이나 매콤한 요리, 간단한 핑거푸드",매운음식 해산물 sweet디저트
2,"붉은 육류요리, 치즈",육류 에피타이저
3,"붉은 육류요리, 치즈",육류 에피타이저
4,"푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",육류
...,...,...
892,"치킨 커틀렛, 치즈 리조또, 숙성이 잘 된 치즈",육류 양식 에피타이저
893,"달콤한 육포, 스테이크, 훈연 소시지, 치즈",육류 에피타이저
894,"양갈비, 로스트비프, 트러플 치즈 파니니, 머쉬룸 파스타",육류 양식 에피타이저
895,"치즈, 육즙이 풍부한 스테이크, 양고기, 블루 치즈",육류 에피타이저


In [72]:
wine_df_final = pd.concat([wine_df,dish_df[['곁들일 음식','음식']]], axis = 1)
wine_df_final

,와인이름,영어이름,와인타입,"나라, 와이너리",포도종,도수,당도,산도,바디,탄닌,와이너리이름,와이너리설명,와이너리특징,아로마 딕셔너리,곁들일 음식,음식
0,이기갈 꼬뜨 뒤 론 레드,E.Guigal Cotes du Rhone Rouge,레드 와인,프랑스 > 론 / 이 기갈,"시라 49%, 그르나쉬 48%, 무르베드르 3%",13.0,1,2,4,1.0,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...","{'빨간 과일': ['딸기'], '허베이셔스': ['풀']}","차가운 육류요리나 가금류, 붉은육류요리, 치즈",육류 에피타이저
1,이기갈 꼬뜨 뒤 론 화이트,E.Guigal Cotes du Rhone Blanc,화이트 와인,프랑스 > 론 / 이 기갈,"비오니에 70%, 루산느 10%, 마르산느 9%, 끌레레뜨 8%, 부르불랑 3%",13.0,1,3,3,NaN,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...","{'꽃': ['아카시아'], '핵과': ['복숭아', '살구']}","해산물이나 매콤한 요리, 간단한 핑거푸드",매운음식 해산물 sweet디저트
2,이기갈 꼬뜨 로띠 ‘브륀 & 블롱드’,E.Guigal Cot-Rotie ‘Brune & Blonde de Guigal’,레드 와인,프랑스 > 론 / 이 기갈,"시라 96%, 비오니에 4%",13.0,1,3,5,1.0,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...","{'자연(Earthy)': ['커피', '토스트'], '오크(Oak)': ['바닐라...","붉은 육류요리, 치즈",육류 에피타이저
3,이기갈 꼬뜨 로띠 ‘샤또 당퓌’,E.Guigal Cote-Rotie ‘Chateau d’Ampuis’,레드 와인,프랑스 > 론 / 이 기갈,"시라 93%, 비오니에 7%",13.0,1,3,5,1.0,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...","{'오크(Oak)': ['바닐라'], '녹색 과일': ['배'], '빨간 과일': ...","붉은 육류요리, 치즈",육류 에피타이저
4,이기갈 꽁드리유,E.Guigal Condrieu,화이트 와인,프랑스 > 론 / 이 기갈,비오니에 100%,13.0,1,3,4,NaN,이 기갈 E.Guigal,이기갈(E.Guigal)은 오늘날 프랑스 론 최고의 생산자로 꼽힌다. 1946년 에...,"""King of the Rhone"" - 영국의 와인전문지 <디캔터(Decanter)...","{'핵과': ['복숭아', '살구']}","푸아그라, 송로버섯을 가미한 달걀요리 등 섬세한 풍미의 음식",육류
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,까테나 자파타 까테나 알타 까베르네 소비뇽,Catena Zapata Catena Alta Cabernet Sauvignon,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,까베르네 소비뇽 100%,NaN,1,3,5,1.0,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,"{'자연(Earthy)': ['토스트'], '오크(Oak)': ['바닐라'], '빨...","치킨 커틀렛, 치즈 리조또, 숙성이 잘 된 치즈",육류 양식 에피타이저
893,까테나 자파타 까테나 알타 말벡,Catena Zapata Catena Alta Malbec,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말백 100%,14.0,1,3,4,1.0,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,"{'오크(Oak)': ['바닐라'], '빨간 과일': ['베리'], '까만 과일':...","달콤한 육포, 스테이크, 훈연 소시지, 치즈",육류 에피타이저
894,까테나 자파타 니카시아 말벡,Catena Zapata Nicasia Malbec,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,14.0,1,3,5,1.0,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,"{'오크(Oak)': ['초콜릿'], '꽃': ['바이올렛'], '빨간 과일': [...","양갈비, 로스트비프, 트러플 치즈 파니니, 머쉬룸 파스타",육류 양식 에피타이저
895,까테나 자파타 아르헨티노 말벡,Catena Zapata Argentino Malbec,레드 와인,아르헨티나 > 멘도자 / 까테나 자파타,말벡 100%,14.0,1,4,5,1.0,까테나 자파타 Catena Zapata,까테나 자파타(Catena Zapata)는 아르헨티나 멘도자의 와인을 전 세계에 제...,아르헨티나 고급 와인의 선구자\r멘도자를 세계적인 와인 산지로 올려 놓은 와이너리\...,"{'녹색 과일': ['배'], '빨간 과일': ['라즈베리', '베리'], '까만 ...","치즈, 육즙이 풍부한 스테이크, 양고기, 블루 치즈",육류 에피타이저


In [73]:
wine_df_final.to_pickle('wine_df_semi_final.pkl')